# 1 Imports

In [8]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from openai import OpenAI
from typing import List
import gradio as gr

# 2 Connect to OpenAI

In [9]:
# Load environment variables in a file called .env

load_dotenv()
api_key = os.getenv('OPEN_API_KEY')

In [10]:
openai = OpenAI()

# 3 OpenAI Model Configuration

In [11]:
system_message = "You are an helpful assistant"

In [ ]:
def message_gpt(user_prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    completion = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
    )

    return completion.choices[0].message.content

In [13]:
message_gpt("What is today's date?")

"Today's date is October 2, 2023."

In [46]:
def stream_gpt(user_prompt, model='gpt-4o-mini'):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt}
    ]

    stream = openai.chat.completions.create(
        model = model,
        messages = messages,
        stream = True
    )
    result = ''
    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

## 4.1 Basic UI

# 4 UI with Gradio

In [14]:
def shout(text):
    return text.upper()

In [15]:
shout('hello')

'HELLO'

In [ ]:
view = gr.Interface(fn = shout, inputs = 'textbox', outputs = 'textbox', flagging_mode= 'never')
view.launch()
#view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [27]:
view = gr.Interface(
    fn = message_gpt,
    inputs = [gr.Textbox(label='Your message:', lines=6)],
    outputs = [gr.Markdown(label='Response:')],
    flagging_mode = 'never'
)

view.launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


## 4.2 Streaming UI

In [47]:
view = gr.Interface(
    fn = stream_gpt,
    inputs = [gr.Textbox(label='Your message:', lines=6)],
    outputs = [gr.Markdown(label='Response:')],
    flagging_mode = 'never'
)

view.launch()

* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


## 4.3 Streaming UI with Model Selection

In [53]:
def stream_model(user_prompt, model):

    if model == 'GPT-4o-mini':
        result = stream_gpt(user_prompt, 'gpt-4o-mini')
    elif model == 'GPT-3.5-Turbo':
        result = stream_gpt(user_prompt, 'gpt-3.5-turbo')
    else:
        raise ValueError('Unknown model')

    for chunk in result:
        yield chunk

In [55]:
view = gr.Interface(
    fn = stream_model,
    inputs = [gr.Textbox(label='Your message:', lines=1), gr.Dropdown(['GPT-4o-mini','GPT-3.5-Turbo'], label = 'Select Model')],
    outputs = [gr.Markdown(label='Response:')],
    flagging_mode = 'never'
)

view.launch()

* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


# 5 Building a company brochure Generator

In [58]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        for irrelevant in soup.body(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.get_text(separator='\n', strip=True)

    def get_contents(self):
        return f"Webpage Title:\n {self.title} \n Webpage Contents: \n {self.text} \n\n"

In [59]:
system_prompt = 'You are an assistant that analyzes the contents of a company website landing page\
    and creates a short brochure about the company for prospective customers, investors, and recruits. Respond in markdown.'

In [65]:
def stream_brochure(company_name, url, model):

    user_prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    user_prompt += Website(url).get_contents()

    if model == 'GPT-4o-mini':
        result = stream_gpt(user_prompt, 'gpt-4o-mini')
    elif model == 'GPT-3.5-Turbo':
        result = stream_gpt(user_prompt, 'gpt-3.5-turbo')
    else:
        raise ValueError('Unknown model')

    for chunk in result:
        yield chunk

In [66]:
view = gr.Interface(
    fn = stream_brochure,
    inputs = [
        gr.Textbox(label='Company Name:', lines=1),
        gr.Textbox(label='Landing page URL:'),
        gr.Dropdown(['GPT-4o-mini','GPT-3.5-Turbo'], label = 'Select Model')],
    outputs = [gr.Markdown(label='Brochure:')],
    flagging_mode = 'never'
)

view.launch()

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.
